In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error,root_mean_squared_error
import os


In [ ]:
# data = pd.read_csv('preprocessed_data.csv')
# data["datetime"]= data["UnixTime"].apply(lambda x: datetime.fromtimestamp(x))
# data

In [ ]:
df = pd.read_csv("sorted_data_final.csv")
df = df.sort_values(by='UnixTime')

#df.drop(columns="DateKey", inplace= True)
df['SinHour'] = np.sin(2 * np.pi * df['Hour'] / 24)
df['CosHour'] = np.cos(2 * np.pi * df['Hour'] / 24)
df.drop(columns=["Month","Day", "Hour" ,"UnixTime"])

use_multivariate = True
seq_length = 48
target_column_name = "Irradiance"

train_ratio = 0.8
train_split_idx = int(len(df) * train_ratio)

scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(df.iloc[:train_split_idx + seq_length])
scaled_test = scaler.transform(df.iloc[train_split_idx + seq_length:])
df_scaled = pd.DataFrame(np.vstack([scaled_train, scaled_test]), columns=df.columns)


def create_sequences(data, target_column, seq_length=12, univariate=False):
    X, y = [], []
    for i in range(len(data) - seq_length):
        if univariate:
            X.append(data[i:i + seq_length, target_column].reshape(-1, 1))
        else:
            X.append(data[i:i + seq_length, :])
        y.append(data[i + seq_length, target_column])
    return np.array(X), np.array(y)

target_column = df_scaled.columns.get_loc(target_column_name)
X, y = create_sequences(df_scaled.values, target_column, seq_length, univariate=not use_multivariate)

X_train, X_test = X[:train_split_idx], X[train_split_idx:]
y_train, y_test = y[:train_split_idx], y[train_split_idx:]

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])

# --- GRU MODEL ---
# model = Sequential([
#     GRU(128, return_sequences=True, input_shape=(seq_length, X_train.shape[2]), kernel_regularizer=l2(0.0005)),
#     GRU(128, return_sequences=True, kernel_regularizer=l2(0.0005)),
#     #GRU(64, return_sequences=False, kernel_regularizer=l2(0.0005)),
#     Dense(64, activation='relu'),
#     Dense(32, activation='relu'),
#     Dropout(0.2),
#     Dense(1)
# ])

model = Sequential([
    LSTM(256, return_sequences=True, input_shape=(seq_length, X_train.shape[2]), kernel_regularizer=l2(0.0001)),
    LSTM(128, return_sequences=True, kernel_regularizer=l2(0.0001)),
    LSTM(64, return_sequences=False, kernel_regularizer=l2(0.0001)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.005), loss='mse')
model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint('best_gru_model.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',       
    factor=0.5,               
    patience=7,             
    min_lr=1e-5,           
    verbose=1
)

history = model.fit(X_train, y_train, epochs=10, batch_size=32,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping, checkpoint])

In [ ]:
y_pred = model.predict(X_test)

def inverse_transform_target(scaled_target, scaler, n_features, target_index):
    zero_filled = np.zeros((len(scaled_target), 13))
    zero_filled[:, target_index] = scaled_target.flatten()
    return scaler.inverse_transform(zero_filled)[:, target_index]

y_test_actual = inverse_transform_target(y_test, scaler, df.shape[1], target_column)
y_pred_actual = inverse_transform_target(y_pred, scaler, df.shape[1], target_column)

mse = mean_squared_error(y_test_actual, y_pred_actual)
r2 = r2_score(y_test_actual, y_pred_actual)
mae = mean_absolute_error(y_test_actual, y_pred_actual)
mape = mean_absolute_percentage_error(y_test_actual, y_pred_actual)
rmse = np.sqrt(mse)

print(f"MSE: {mse}")
print(f"R2  {r2}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}")
print(f"RMSE: {rmse}")




#this visualisation was done by chat gpt weekly graphs wanted to make sure that the model is behaving correctly
# and I did not see small time frames very well if I did it on whole test set
df['Timestamp'] = pd.to_datetime(df['UnixTime'], unit='s')
df['ISO_Week'] = df['Timestamp'].dt.isocalendar().week

# Extract test index range
test_start_idx = train_split_idx + seq_length
test_end_idx = test_start_idx + len(y_test)

# Get timestamps and weeks for test set
timestamps = df.iloc[test_start_idx:test_end_idx]['Timestamp'].values
iso_weeks = df.iloc[test_start_idx:test_end_idx]['ISO_Week'].values

# Build aligned predictions DataFrame
predictions_df = pd.DataFrame({
    'Timestamp': timestamps,
    'Week': iso_weeks,
    'Predicted Irradiance': y_pred_actual,
    'Actual Irradiance': y_test_actual
})

# Make sure the output directory exists
os.makedirs("weekly_plots", exist_ok=True)

# Plot each week in the test set
for week in sorted(predictions_df['Week'].unique()):
    weekly_data = predictions_df[predictions_df['Week'] == week]
    
    plt.figure(figsize=(10, 5))
    plt.plot(weekly_data['Timestamp'], weekly_data['Actual Irradiance'], label="Skutočnosť", color='blue', alpha=0.6)
    plt.plot(weekly_data['Timestamp'], weekly_data['Predicted Irradiance'], label="Predikcia", color='red', alpha=0.6)
    plt.xlabel("Čas")
    plt.ylabel("Slnečné žiarenie")
    plt.title(f"Tyždeň {week} - Predikcia vs Skutočnosť")
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"weekly_plots/week_{week}.png", dpi=300)
    plt.close()


plt.figure(figsize=(8, 8))
plt.scatter(y_test_actual, y_pred_actual, alpha=0.5, color='green', edgecolors='k')
plt.plot([min(y_test_actual), max(y_test_actual)],
         [min(y_test_actual), max(y_test_actual)],
         color='black', linestyle='--', linewidth=2)

plt.xlabel("Skutočná hodnota (Irradiancia)")
plt.ylabel("Predikovaná hodnota (Irradiancia)")
plt.title(f"R² Rozptylový graf\nR² skóre: {r2:.3f}")
plt.grid(True)
plt.tight_layout()

# Ulož graf
plt.savefig("r2_scatter_plot.png", dpi=300)
plt.close()